# Model Inference

## Import Libraries

In [407]:
import pandas as pd
import numpy as np
import tensorflow as tf
import pickle
import json

from tensorflow.keras.models import load_model
from sklearn.metrics import mean_squared_error, mean_absolute_error



## Load Model

In [196]:
# Load data model
with open('minmaxScaler (2).pkl', 'rb') as file_1:
    scaler = pickle.load(file_1)

model_timeseries = load_model('timeseries_model (2).h5')

## Create new data

In [412]:
# create data
data_inf = {
    'week_number': ['2023-14', '2023-14', '2023-14', '2023-14', '2023-14', '2023-14', '2023-14'],
    'week_start_date': ['2023-04-03', '2023-04-03', '2023-04-03', '2023-04-03', '2023-04-03', '2023-04-03', '2023-04-03'],
    'week_end_date': ['2023-04-09', '2023-04-09', '2023-04-09', '2023-04-09', '2023-04-09', '2023-04-09', '2023-04-09'], 
    'product_item': ['product_297', 'product_646', 'product_298', 'product_647', 'product_648', 'product_299', 'product_649'],
    'quantity': [21872, 75, 436, 907, 81, 42920, 53],
}

data_inf = pd.DataFrame(data_inf)

In [413]:
# Preprocessing data_inf
data_inf['week_start_date'] = pd.to_datetime(data_inf['week_start_date'])
data_inf['week_end_date'] = pd.to_datetime(data_inf['week_end_date'])

In [414]:
# menambahkan waktu 7 hari
next_week_start_date = data_inf['week_start_date'] + pd.Timedelta(days=7)
next_week_end_date = data_inf['week_end_date'] + pd.Timedelta(days=7)

In [415]:
data_inf['week_start_date'] = pd.to_datetime(data_inf['week_start_date'])
data_inf = data_inf.set_index('week_start_date')

In [416]:
# konversi week_start_date dan week_end_date menjadi week_number
next_week_number = next_week_start_date[0].week

In [417]:
# hapus kolom week_number dari data_inf
data_inf.drop(columns=['week_number'], inplace=True)

In [418]:
# tambahkan kolom week_number baru ke data_inf
data_inf['week_number'] = next_week_number

In [438]:
data_inf

,week_end_date,product_item,quantity,week_number
week_start_date,,,,
2023-04-03,2023-04-09,297,21872,14
2023-04-03,2023-04-09,646,75,14
2023-04-03,2023-04-09,298,436,14
2023-04-03,2023-04-09,647,907,14
2023-04-03,2023-04-09,648,81,14
2023-04-03,2023-04-09,299,42920,14
2023-04-03,2023-04-09,649,53,14


In [419]:
# Menghapus 'product_' dari setiap nilai dalam kolom product_item
data_inf['product_item'] = data_inf['product_item'].str.replace('product_', '') 
data_inf['product_item'] = data_inf['product_item'].astype(int)

In [420]:
# buat input window dari data_inf
window_size = 7
X_inf = data_inf[['week_number', 'product_item', 'quantity']].values
X_inf_windowed = X_inf.reshape(1, window_size, X_inf.shape[1])
X_inf_windowed = X_inf_windowed.astype('float32')

In [421]:
# Define input variables for LSTM
X_inf = data_inf[['week_number', 'product_item', 'quantity']].values

In [439]:
# prediksi 7 hari ke depan
for i in range(7):
    # Melakukan prediksi untuk minggu selanjutnya
    y_pred = model_timeseries.predict(X_inf_windowed)
    
    # Menambahkan data prediksi ke dalam dataframe
    next_week_start_date = next_week_start_date + pd.Timedelta(days=7)
    next_week_end_date = next_week_end_date + pd.Timedelta(days=7)
    next_week_end_date = next_week_start_date + pd.Timedelta(days=6)
    next_week_start_date_str = next_week_start_date.strftime('%Y-%m-%d')
    next_week_end_date_str = next_week_end_date.strftime('%Y-%m-%d')
    new_row = [next_week_number, next_week_start_date_str, next_week_end_date_str, int(data_inf['product_item'].iloc[-1]), y_pred[0, -1]]
    new_row = [next_week_end_date_str, int(data_inf['product_item'].iloc[-1]), y_pred[0, -1], next_week_number]
    data_inf = data_inf.append(pd.DataFrame([new_row], columns=['week_end_date', 'product_item', 'quantity', 'week_number']).set_index('week_end_date'))


    
    # Memperbarui windowed input
    X_inf = data_inf[['week_number', 'product_item', 'quantity']].values
    X_inf_windowed = X_inf[-window_size:].reshape(1, window_size, X_inf.shape[1])


1/1 [==============================] - 0s 81ms/step


/usr/local/lib/python3.9/dist-packages/tensorflow/python/data/ops/structured_function.py:254: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


1/1 [==============================] - 0s 67ms/step


<ipython-input-439-dda9c6880290>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_inf = data_inf.append(pd.DataFrame([new_row], columns=['week_end_date', 'product_item', 'quantity', 'week_number']).set_index('week_end_date'))
/usr/local/lib/python3.9/dist-packages/tensorflow/python/data/ops/structured_function.py:254: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
<ipython-input-439-dda9c6880290>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_inf = data_inf.append(pd.DataFrame([new_row], columns=['week_end_date', 'product_item', 'quantity', 'week_number']).set_index('week_end_date'))
/usr/

1/1 [==============================] - 0s 66ms/step


<ipython-input-439-dda9c6880290>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_inf = data_inf.append(pd.DataFrame([new_row], columns=['week_end_date', 'product_item', 'quantity', 'week_number']).set_index('week_end_date'))
/usr/local/lib/python3.9/dist-packages/tensorflow/python/data/ops/structured_function.py:254: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
<ipython-input-439-dda9c6880290>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_inf = data_inf.append(pd.DataFrame([new_row], columns=['week_end_date', 'product_item', 'quantity', 'week_number']).set_index('week_end_date'))
/usr/

1/1 [==============================] - 0s 68ms/step


<ipython-input-439-dda9c6880290>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_inf = data_inf.append(pd.DataFrame([new_row], columns=['week_end_date', 'product_item', 'quantity', 'week_number']).set_index('week_end_date'))
/usr/local/lib/python3.9/dist-packages/tensorflow/python/data/ops/structured_function.py:254: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
<ipython-input-439-dda9c6880290>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_inf = data_inf.append(pd.DataFrame([new_row], columns=['week_end_date', 'product_item', 'quantity', 'week_number']).set_index('week_end_date'))
/usr/

1/1 [==============================] - 0s 94ms/step


<ipython-input-439-dda9c6880290>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_inf = data_inf.append(pd.DataFrame([new_row], columns=['week_end_date', 'product_item', 'quantity', 'week_number']).set_index('week_end_date'))


In [446]:
# Define window size
window_size = 7

# Define function to create input windows
def create_input_windows(X, y, window_size):
    X_windowed = []
    y_windowed = []
    for i in range(window_size, len(X)):
        X_windowed.append(X[i-window_size:i])
        y_windowed.append(y[i-window_size:i+1])
    return np.array(X_windowed), np.array(y_windowed)

In [447]:
# Create input windows for inference
X_inf_windowed, _ = create_input_windows(X_inf, np.zeros(X_inf.shape[0]), window_size=7)
X_inf_windowed = X_inf_windowed.reshape(X_inf_windowed.shape[0], window_size, X_inf_windowed.shape[2])

# Predict quantity for the next 7 days using the trained model
y_inf_pred = model_timeseries.predict(X_inf_windowed)

# Print the predicted quantities for the next 7 days
print('Predicted quantities for the next 7 days:')
print(y_inf_pred[:,-1])

1/1 [==============================] - 0s 77ms/step
Predicted quantities for the next 7 days:
[-0.00069542 -0.00300451 -0.00383235 -0.00159359  0.00023478  0.00066616
  0.00079078]


# Kesimpulan

- penjualan tertinggi berada pada bulan september 2022, dan setelah itu berada di bulan juni 2022
dibulan2 lainnya rata2 penjualan sama.

- penjualan produk terbanyak sebesar 18206376 unit, penjualan ke 2 sebanyak 9122007 unit, dan untuk yg ke 3 sebanyak 6771207 unit

- Terdapat 102733 transaksi yang tercatat pada data tersebut.

- Rata-rata jumlah produk yang terjual per transaksi adalah 3191.56 unit, dengan standar deviasi sebesar 12074.63 unit.

- Jumlah produk terjual minimal adalah 0 unit, sedangkan jumlah produk terjual maksimal mencapai 774732 unit.

- Kuartil pertama (Q1) pada jumlah produk terjual adalah 75 unit, median (Q2) adalah 414 unit, dan kuartil ketiga (Q3) adalah 1890 unit.

- Total jumlah produk yang terjual selama periode tersebut adalah 327878782 unit.

- top produk terpopuler adalah product_1, product_30, product_33, product_34 dan product_35

- dari masing2 produk terdapat pola penjualan yg berbeda2

- pada saat model dilakukan improvement,model yg dipilih adalah model sequential, karena memiliki beragam pola dan masih tergolong goodfit

- kelebihan dari model ini adalah, model ini dapat memprediksi 7 hari kedepan per product_item, namun, kelemahanya adalah, karena scrore MSEnya 3.7205 ada kemungkinan untuk salah prediksi, diperlukan improvisasi lagi agar score MSE nya lebih rendah